In [1]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()
bert_model.to('cpu')  # Assuming you are using a CUDA-enabled device

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to('cpu') for key, value in inputs.items()}
    with torch.no_grad():
        outputs = bert_model(**inputs)
    # Use the pooled output for a single vector per input sequence
    return outputs.last_hidden_state[:, 0, :]  # Get the [CLS] token's embeddings

/Users/deric/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/deric/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/deric/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.unsqueeze(0)  # Ensure there's a batch dimension
        _, (hn, _) = self.lstm(x)
        return self.fc(hn.squeeze(0))

# Load your trained model
def load_lstm_model(model_path, device='cpu'):
    model = LSTMClassifier(embedding_dim=768, hidden_dim=128, output_dim=2)  # Adjust dimensions as necessary
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.to(device)
    model.eval()
    return model

lstm_model = load_lstm_model('best_model_BERT.pth')

In [3]:
def predict_headline(lstm_model, text):
    embeddings = get_bert_embeddings(text)
    # Ensure embeddings are unsqueezed if not already batched
    if embeddings.dim() == 1:
        embeddings = embeddings.unsqueeze(0)
    output = lstm_model(embeddings)
    # Check output shape
    print("Output shape:", output.shape)
    scores = torch.softmax(output, dim=1).detach().cpu().numpy()
    return scores

# Example Usage
headline = "5 found dead in istanbul after pkk bombing"
scores = predict_headline(lstm_model, headline)
print("Scores:", scores)

Output shape: torch.Size([1, 2])
Scores: [[9.9929690e-01 7.0311996e-04]]
